# Merging for data downloaded from FRED

In [1]:
import pandas as pd
from pandas.tseries.offsets import MonthBegin
import os
from ydata_profiling import ProfileReport

In [2]:
#Folder with the data
data_folder = "raw_data/Macroeconomic Factors/"

In [3]:
# Mapping of FRED filenames to human-readable column names
rename_dict = {
    "BOPGSTB.csv": "Trade Balance",
    "CPIAUCSL.csv": "Inflation (CPI)",
    "CPILFESL.csv": "Core Inflation",
    "FEDFUNDS.csv": "Interest Rate",
    "INDPRO.csv": "Industrial Production",
    "RSAFS.csv": "Retail Sales",
    "UMCSENT.csv": "Consumer Sentiment",
    "UNRATE.csv": "Unemployment Rate",
}


In [4]:
# Initialize merged_df
merged_df = None

for filename, pretty_name in rename_dict.items():
    filepath = os.path.join(data_folder, filename)
    df = pd.read_csv(filepath)
    
    # Rename columns
    df = df.rename(columns={
        "observation_date": "DATE",
        df.columns[1]: pretty_name  # the second column has the FRED code
    })
    
    df["DATE"] = pd.to_datetime(df["DATE"])
    
    if merged_df is None:
        merged_df = df
    else:
        merged_df = pd.merge(merged_df, df, on="DATE", how="inner")

In [5]:
# Final touches
merged_df = merged_df.sort_values("DATE").set_index("DATE")

In [6]:
# Preview the result
merged_df.head()

,Trade Balance,Inflation (CPI),Core Inflation,Interest Rate,Industrial Production,Retail Sales,Consumer Sentiment,Unemployment Rate
DATE,,,,,,,,
2002-01-01,-29678,177.7,188.7,1.73,88.4634,283508,93.0,5.7
2002-02-01,-32501,178.0,189.1,1.74,88.4578,285054,90.7,5.7
2002-03-01,-31161,178.5,189.2,1.73,89.1265,284262,95.7,5.7
2002-04-01,-33965,179.3,189.7,1.75,89.5507,288833,93.0,5.9
2002-05-01,-34026,179.5,190.0,1.75,89.9348,284951,96.9,5.8


# Merging files with different dates

In [7]:
new_macro_files = {
    "NAPM.csv": "Manufacturing PMI",
    "SP500Historical.csv": "S&P 500 Index",
    "VIX Index.csv": "VIX Index"
}

In [8]:
# Initialize
extra_macro_df = None

for file, colname in new_macro_files.items():
    filepath = os.path.join(data_folder, file)
    df = pd.read_csv(filepath)

    # Just rename columns and fix date — keep everything else as is
    df.columns = df.columns.str.strip()  # In case of space-padding
    df = df.rename(columns={"Date": "DATE", "PX_LAST": colname})

    # Push date to 1st of next month
    df["DATE"] = pd.to_datetime(df["DATE"]) + MonthBegin(1)

    if extra_macro_df is None:
        extra_macro_df = df
    else:
        extra_macro_df = pd.merge(extra_macro_df, df, on="DATE", how="outer")

In [9]:
# Final touches
extra_macro_df = extra_macro_df.sort_values("DATE").set_index("DATE")

In [10]:
extra_macro_df.head()

,Manufacturing PMI,S&P 500 Index,VIX Index
DATE,,,
2002-02-01,47.5,1130.21,21.09
2002-03-01,50.7,1106.73,21.59
2002-04-01,52.4,1147.39,17.40
2002-05-01,52.4,1076.92,21.91
2002-06-01,53.1,1067.14,19.98


# Merging both the dfs

In [11]:
# Merge on the date index
macro_df = pd.merge(merged_df, extra_macro_df, left_index=True, right_index=True, how="outer")

# Sort by date and save
macro_df = macro_df.sort_index()
macro_df.to_csv("macro_data.csv")


In [12]:
macro_df.head()

,Trade Balance,Inflation (CPI),Core Inflation,Interest Rate,Industrial Production,Retail Sales,Consumer Sentiment,Unemployment Rate,Manufacturing PMI,S&P 500 Index,VIX Index
DATE,,,,,,,,,,,
2002-01-01,-29678.0,177.7,188.7,1.73,88.4634,283508.0,93.0,5.7,NaN,NaN,NaN
2002-02-01,-32501.0,178.0,189.1,1.74,88.4578,285054.0,90.7,5.7,47.5,1130.21,21.09
2002-03-01,-31161.0,178.5,189.2,1.73,89.1265,284262.0,95.7,5.7,50.7,1106.73,21.59
2002-04-01,-33965.0,179.3,189.7,1.75,89.5507,288833.0,93.0,5.9,52.4,1147.39,17.40
2002-05-01,-34026.0,179.5,190.0,1.75,89.9348,284951.0,96.9,5.8,52.4,1076.92,21.91


### No need to run the cell below, you should be able to see the html report uploaded in the repo

In [13]:
# Load final macroeconomic dataset
# df = pd.read_csv("macro_data.csv", parse_dates=["DATE"])
# df = df.set_index("DATE")  

# Generate profiling report
# profile = ProfileReport(df, 
#                         title="Macroeconomic EDA Report",
#                         html={"style": {"full_width": True}}, 
#                         minimal=False)

# Save as HTML
# profile.to_file("macro_eda_report.html")


100%|████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 2198.38it/s]
